<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/application_model_zoo/Example%20-%20Road%20Segmentation%20in%20Satellite%20Images%20-%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents


## 1. Installation Instructions



## 2. Use trained model to Roads in satellite imagery



## 3. How to train a custom segmenter using "Road Segmentation Dataset"

# About the networks


1. UNet
  - https://arxiv.org/abs/1505.04597
  - https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47
  - https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5


2. FPN
  - http://openaccess.thecvf.com/content_cvpr_2017/papers/Lin_Feature_Pyramid_Networks_CVPR_2017_paper.pdf
  - https://towardsdatascience.com/review-fpn-feature-pyramid-network-object-detection-262fc7482610
  - https://medium.com/@jonathan_hui/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c


3. PSPNet
  - https://arxiv.org/abs/1612.01105
  - https://towardsdatascience.com/review-pspnet-winner-in-ilsvrc-2016-semantic-segmentation-scene-parsing-e089e5df177d
  - https://developers.arcgis.com/python/guide/how-pspnet-works/


4. Linknet
  - https://arxiv.org/pdf/1707.03718.pdf
  - https://neptune.ai/blog/image-segmentation-tips-and-tricks-from-kaggle-competitions

# Installation

 - Run these commands
     
     - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git
     
     - cd Monk_Object_Detection/9_segmentation_models/installation
     
 - Select the right requirements file and run
 
     - cat requirements_cuda9.0.txt | xargs -n 1 -L 1 pip install

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
# For colab use the command below
! cd Monk_Object_Detection/9_segmentation_models/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install


# For Local systems and cloud select the right CUDA version
#! cd Monk_Object_Detection/9_segmentation_models/installation && cat requirements_cuda10.0.txt | xargs -n 1 -L 1 pip install

# Use already trained model for demo

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from infer_segmentation import Infer

In [ ]:
gtf = Infer();

In [ ]:
classes_dict = {
    'background': 0, 
    'road': 1
};
classes_to_train = ['background', 'road'];

In [ ]:
gtf.Data_Params(classes_dict, classes_to_train, image_shape=[384, 384])

In [ ]:
# Download trained model

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1WF7ByC_rw70sGB_YyKd4XnB6GEErs9W4' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1WF7ByC_rw70sGB_YyKd4XnB6GEErs9W4" -O seg_road_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq seg_road_trained.zip

In [ ]:
gtf.Model_Params(model="Unet", backbone="efficientnetb3", path_to_model='seg_road_trained/best_model.h5')

In [ ]:
gtf.Setup();

In [ ]:
ls seg_road_trained/test

In [ ]:
gtf.Predict("seg_road_trained/test/1.png", vis=True);

In [ ]:
gtf.Predict("seg_road_trained/test/2.png", vis=True);

In [ ]:
gtf.Predict("seg_road_trained/test/3.png", vis=True);

In [ ]:
gtf.Predict("seg_road_trained/test/4.png", vis=True);

In [ ]:
gtf.Predict("seg_road_trained/test/5.png", vis=True);

In [ ]:
gtf.Predict("seg_road_trained/test/6.png", vis=True);

In [ ]:
gtf.Predict("seg_road_trained/test/7.png", vis=True);

In [ ]:
gtf.Predict("seg_road_trained/test/8.png", vis=True);

# Train you own detector

# Monk Format

## Dataset Directory Structure

      root_dir
          |
          | 
          |         
          |----train_img_dir
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on) 
          |
          |----train_mask_dir
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)
          |
          |----val_img_dir (optional)
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)
          |
          |----val_mask_dir (optional)
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)

          
          

# Sample Dataset Credits

    credits: https://www.kaggle.com/franciscoescobar/satellite-images-of-water-bodies

In [ ]:
! pip install kaggle

In [ ]:
! kaggle datasets download srikaranand/road-segmentation-dataset

In [ ]:
! unzip -qq road-segmentation-dataset.zip

In [ ]:
! mkdir dataset
! mkdir dataset/images
! mkdir dataset/masks

In [ ]:
import os
import cv2
import numpy as np 
from tqdm import tqdm

In [ ]:
img_list = os.listdir("data/training/groundtruth");

In [ ]:
for i in tqdm(range(len(img_list))):
    img = cv2.imread("data/training/images/" + img_list[i]);
    img = cv2.resize(img, (384, 384));
    cv2.imwrite("dataset/images/" + str(i) + ".png", img);

    mask = cv2.imread("data/training/groundtruth/" + img_list[i], 0);
    mask = cv2.resize(mask, (384, 384));
    mask[mask > 0] = 1;
    cv2.imwrite("dataset/masks/" + str(i) + ".png", img);

## Training 

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from train_segmentation import Segmenter

In [ ]:
gtf = Segmenter();

In [ ]:
train_img_dir = "dataset/images";
train_mask_dir = "dataset/masks";

val_img_dir = "dataset/images";
val_mask_dir = "dataset/masks";


In [ ]:
classes_dict = {
    'background': 0, 
    'road': 1
};
classes_to_train = ['background', 'road'];

In [ ]:
gtf.Train_Dataset(train_img_dir, train_mask_dir, classes_dict, classes_to_train)

In [ ]:
gtf.Val_Dataset(val_img_dir, val_mask_dir)

In [ ]:
gtf.List_Backbones();

In [ ]:
gtf.Data_Params(batch_size=2, backbone="efficientnetb3", image_shape=[384, 384])

In [ ]:
gtf.List_Models();

In [ ]:
gtf.Model_Params(model="Unet")

In [ ]:
gtf.Train_Params(lr=0.0001)

In [ ]:
gtf.Setup();

In [ ]:
gtf.Train(num_epochs=300);

In [ ]:
gtf.Visualize_Training_History();

# Inference

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from infer_segmentation import Infer

In [ ]:
gtf = Infer();

In [ ]:
classes_dict = {
    'background': 0, 
    'road': 1
};
classes_to_train = ['background', 'road'];

In [ ]:
gtf.Data_Params(classes_dict, classes_to_train, image_shape=[384, 384])

In [ ]:
gtf.Model_Params(model="Unet", backbone="efficientnetb3", path_to_model='best_model.h5')

In [ ]:
gtf.Setup();

In [ ]:
import os
img_list = os.listdir("dataset/images/")

In [ ]:
gtf.Predict("dataset/images/" + img_list[i], vis=True);